#### 1. Install Dependencies

In [ ]:
pip install -r ../requirements.txt -q

#### 2. Verify python-dotenv

In [ ]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

#### 3. Chat Bot Without Memory

In [ ]:
# simple chatbot with no memory i.e no context between chat messages
# each question is a new question and the model has no memory of previous questions
# for eg: 
# __________________________________________________
# input: 10 + 10 is
# resp: 20
# __________________________________________________
# input: times 2 is
# resp: Oh, if you multiply something by 2, you are essentially doubling it.
# __________________________________________________

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

# init
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
prompt = ChatPromptTemplate(
  input_variables = ['content'],
  messages = [
    SystemMessage(content = 'You are a chat bot having a conversation with a human'),
    SystemMessage(content = 'Respond like how a human would and add details if applicable'),
    HumanMessagePromptTemplate.from_template('{content}'),
  ]
)
chain = LLMChain(llm = llm, prompt = prompt, verbose = False)

# infinite loop
while True:
  
  # get user input
  content = input('your prompt: ')
  
  # break if user wants to quit
  if content in ['quit', 'exit', 'bye', 'q']:
    print('Goodbye!')
    break
  
  # else ask the model for a response
  resp = chain.run({'content': content})
  print(f'input: {content}')
  print(f'resp: {resp}')
  print('_' * 50)

#### 4. Chat Bot With Memory (but no persistence i.e. sessions are NOT saved)

In [ ]:
# simple chatbot with memory i.e it saves context between chat messages
# for eg: 
# __________________________________________________
# input: 10 + 10 is
# resp: 20
# __________________________________________________
# input: times 2 is
# resp: 40
# __________________________________________________

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# init
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

# memory_key = a key where chat messages are stored
# return_messages = true, means that the memory will return the chat messages as a list
memory = ConversationBufferMemory(
  memory_key = 'chat_session', 
  return_messages = True
)

prompt = ChatPromptTemplate(
  input_variables = ['content'],
  messages = [
    SystemMessage(content = 'You are a chat bot having a conversation with a human'),
    SystemMessage(content = 'Respond like how a human would and add details if applicable'),
    MessagesPlaceholder(variable_name = 'chat_session'), # where the memory will be stored
    HumanMessagePromptTemplate.from_template('{content}'),
  ]
)
chain = LLMChain(
  llm = llm, 
  prompt = prompt, 
  memory = memory, # add memory to chain
  verbose = False)

# infinite loop
while True:
  
  # get user input
  content = input('your prompt: ')
  
  # break if user wants to quit
  if content in ['quit', 'exit', 'bye', 'q']:
    print('Goodbye!')
    break
  
  # else ask the model for a response
  resp = chain.run({'content': content})
  print(f'input: {content}')
  print(f'resp: {resp}')
  print('_' * 50)

#### 5. Chat Bot With Memory & Saved Sessions

In [ ]:
# simple chatbot with memory & persistence
# for eg: 
# __________________________________________________
# input: 10 + 10 is
# resp: 20
# __________________________________________________
# input: times 2 is
# resp: 40
# __________________________________________________
# quit and re-run program
# __________________________________________________
# input: plus 10 is
# resp: 50
# __________________________________________________

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

# init
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

# save chat history to a file
history = FileChatMessageHistory('chat_history.json')

# chat_memory = tell llm to use the chat history. if you run the program again, it will remember prev questions and answer accordingly
# i.e. we now have persistent memory
memory = ConversationBufferMemory(
  memory_key = 'chat_session', 
  chat_memory = history, # add chat history to memory
  return_messages = True
)

prompt = ChatPromptTemplate(
  input_variables = ['content'],
  messages = [
    SystemMessage(content = 'You are a chat bot having a conversation with a human'),
    SystemMessage(content = 'Respond like how a human would and add details if applicable'),
    MessagesPlaceholder(variable_name = 'chat_session'),
    HumanMessagePromptTemplate.from_template('{content}'),
  ]
)
chain = LLMChain(
  llm = llm, 
  prompt = prompt, 
  memory = memory,
  verbose = False)

# infinite loop
while True:
  
  # get user input
  content = input('your prompt: ')
  
  # break if user wants to quit
  if content in ['quit', 'exit', 'bye', 'q']:
    print('Goodbye!')
    break
  
  # else ask the model for a response
  resp = chain.run({'content': content})
  print(f'input: {content}')
  print(f'resp: {resp}')
  print('_' * 50)